<br>
    <center>МИНИСТЕРСТВО НАУКИ И ВЫСШЕГО ОБРАЗОВАНИЯ</center>
    <center>федеральное государственное автономное образовательное учреждение высшего образования </center> <center>«Самарский национальный исследовательский университет имени академика С.П. Королева»</center>
    <center>(Самарский университет)</center> </br>

<br/>

<br>
<center>Институт 	     информатики и кибернетики</center>                                                   	  
<center>Кафедра 	     технической кибернетики</center>                                                              	
</br>
<br/>

<br>

<br/>
<br/>
<br/>
<br>
<center>ОТЧЕТ</center>
<center>по лабораторной работе №2</center>

<center>«Обработка данных с использование Kafka»</center>
<center>(дополнительный код для первого обработчика схемы two consumers launched in a group)</center>
<br/>
<center>по дисциплине <strong>«Большие данные»</strong></center>
<br/>
<center></center>
</br>
<br/>
<br/>
<br/>



<br>
<p style="text-align:right;">Выполнил: Ерофеев Н.В.
<br>6131-010402D
<br>    
<br>Преподаватель: Попов С.Б.
</p>
<br/>
<br/>
<br/>
    <br/>
<br/>
<br/>
<center>Самара 2025</center>


### Экспорт необходимых модулей

In [26]:
from kafka import KafkaConsumer

### Определение списка серверов kafka и имени топика с двумя разделами

In [27]:
BOOTSTRAP_SERVERS = ['localhost:9092']
TOPIC_2 = 'rides2' 

## Реализация схемы с двумя обработчиками (two consumers launched in a group)

### Реализация обработчика данных (consumer)

1. Создаём экземпляр класса KafkaConsumer (не забываем в параметрах определить параметры *group_id*, *consumer_timeout_ms* и десериализаторы для key- и value-составляющих в сообщении). Параметр *consumer_timeout_ms* необходимо определить с учётом количества сообщений и времени задержки при отправке (двух секунд может не хватить, поскольку иногда много сообщений подряд посылаются в один раздел).
2. Создаём массивы для накопления значений trip_count, passenger_count, trip_distance, total_amount с нулевыми значениями и соответствующим типом данных
3. В цикле получения сообщений объекта KafkaConsumer:
   * разделяем получаемую value-строку на отдельные элементы,
   * обновляем значения в массивах-накопителях, используя (key - 1) как индекс соответствующего массива 
4. Распечатываем отдельно для каждого перевозчика:
   * количество поездок(trip_count),
   * число перевезённых пассажиров(passenger_count),
   * расстояние поездок(trip_distance)
   * сумму оплаты(total_amount)
5. Завершаем работу объекта KafkaProducer

In [28]:
consumer1 = KafkaConsumer(TOPIC_2,
                          bootstrap_servers=BOOTSTRAP_SERVERS,
                          group_id ='rides2',
                          auto_offset_reset='earliest',
                          consumer_timeout_ms=60000,  
                          key_deserializer= lambda key: int(key.decode('utf-8')), 
                          value_deserializer= lambda value: value.decode('utf-8'))

In [29]:
print("CONSUMER #1 запущен, ждём сообщения...")

stats = {}
msg_count = 0

try:
    for message in consumer1:   # цикл по приходящим сообщениям
        key = message.key       # VendorID в виде int
        value = message.value   # строка CSV

        row = value.split(',')
        vendor_id = row[0]
        passenger_count = int(row[3])
        trip_distance = float(row[4])
        total_amount = float(row[16])

        if vendor_id not in stats:
            stats[vendor_id] = {
                "trips": 0,
                "passengers": 0,
                "distance": 0.0,
                "amount": 0.0,
            }

        stats[vendor_id]["trips"] += 1
        stats[vendor_id]["passengers"] += passenger_count
        stats[vendor_id]["distance"] += trip_distance
        stats[vendor_id]["amount"] += total_amount

        msg_count += 1
        print(f"partition={message.partition}, offset={message.offset}, vendor={vendor_id}, f={total_amount}")

except KeyboardInterrupt:
    print("Остановка consumer1...")

print("\nИТОГИ CONSUMER #1:")
for v, s in stats.items():
    print(f"VendorID {v}: поездок={s['trips']}, пассажиров={s['passengers']}, "
          f"расстояние={s['distance']:.2f}, сумма={s['amount']:.2f}")
print(f"Всего обработано сообщений: {msg_count}")

consumer1.close()

CONSUMER #1 запущен, ждём сообщения...
partition=1, offset=0, vendor=1, f=9.3
partition=1, offset=1, vendor=1, f=27.8
partition=1, offset=2, vendor=1, f=33.8
partition=1, offset=3, vendor=1, f=28.3
partition=1, offset=4, vendor=1, f=5.8
partition=1, offset=5, vendor=1, f=39.8
partition=1, offset=6, vendor=1, f=30.8
partition=1, offset=7, vendor=1, f=11.3
partition=1, offset=8, vendor=1, f=10.8
partition=1, offset=9, vendor=1, f=8.8
partition=1, offset=10, vendor=1, f=15.8
partition=1, offset=11, vendor=1, f=10.8
partition=1, offset=12, vendor=1, f=11.15
partition=1, offset=13, vendor=1, f=12.95
partition=1, offset=14, vendor=1, f=12.3
partition=1, offset=15, vendor=1, f=14.75
partition=1, offset=16, vendor=1, f=11.3
partition=1, offset=17, vendor=1, f=13.8
partition=1, offset=18, vendor=1, f=9.3
partition=1, offset=19, vendor=1, f=9.8
partition=1, offset=20, vendor=1, f=8.3
partition=1, offset=21, vendor=1, f=29.15
partition=1, offset=22, vendor=1, f=44.3
partition=1, offset=23, vendor